In [1]:
import pandas as pd
import numpy as np
import math
from pprint import pprint

In [2]:
%ls

evaluation.ipynb                matching_langchain_complex.ipynb
langchain_0shot_complex/        matching_langchain_simple.ipynb
langchain_0shot_simple/         matching_one_shot_complex.ipynb
matched_complex_oneshot.csv     matching_one_shot_simple.ipynb
matched_complex_zeroshot.csv    matching_zero_shot_complex.ipynb
matched_langchain_complex.csv   matching_zero_shot_simple.ipynb
matched_langchain_simple.csv    schema_0shot_complex/
matched_langchain_zeroshot.csv  schema_0shot_simple/
matched_simple_oneshot.csv      schema_1shot_complex/
matched_simple_zeroshot.csv     schema_1shot_simple/


In [66]:
def find_evaluation_scores(filename):
    hea_fname = '../RMPEA Dataset filtered Aug 30 23.csv'
    hea_df = pd.read_csv(hea_fname)
    filtered_df = hea_df[hea_df['PROPERTY: YS (MPa)'].notnull()]
    simple_schema_matched = pd.read_csv(filename)
    print(simple_schema_matched.columns)
    other_properties_extracted = ['PROPERTY: grain size ($\mu$m)_extracted',
                              'PROPERTY: Exp. Density (g/cm$^3$)_extracted',
                              'PROPERTY: HV_extracted',
                              'PROPERTY: Test temperature ($^\circ$C)_extracted',
                              'PROPERTY: UTS (MPa)_extracted',
                              'PROPERTY: Elongation (%)_extracted',
                              'PROPERTY: Elongation plastic (%)_extracted']
    original_properties = [prop.replace('extracted', 'original') for prop in other_properties_extracted]
    
    list_row_match = []
    
    for i, row in simple_schema_matched.iterrows():
        row_value_extracted = {prop.replace('_extracted', ''):row[prop] for prop in other_properties_extracted}
        row_value_original = {prop.replace('_original', ''):row[prop] for prop in original_properties}
        # print(row_value_original)
        match_dict = {}
        for k_org, v_org in row_value_original.items():
            v_ext = row_value_extracted[k_org]
            try:
                if type(v_ext) == str:
                    if '>' in v_ext or '%' in v_ext:
                        v_ext = v_ext.replace('>', '').replace('%','')
                    v_ext = float(v_ext)
            except Exception as e:
                print(e)
                pass
            if math.isnan(v_org):
                match_dict[k_org] = -1 # no valid entry
                continue
            # elif math.isnan(v_ext)==True and math.isnan(v_org)==False:
            #   match_dict[k_org] = -2 # valid entry present but nothing extracted
        
            elif type(v_ext)!=float or math.isnan(v_ext)==True:
                match_dict[k_org] = -2 # valid entry present but nothing extracted
                continue
            else:
                if v_org==v_ext:
                    match_dict[k_org] = True # match
                elif v_org != v_ext:
                    match_dict[k_org] = False # no match
            list_row_match.append(match_dict)
        
    eval_df = pd.DataFrame(list_row_match)
      
    number_of_rel_items = {}
    number_of_rel_items_retreived = {}
    number_of_retrieved_items = {}
    
    for col in eval_df.columns:
        distribution = eval_df[col].value_counts().to_dict()
        neg_1 = distribution.get(-1)
        neg_2 = distribution.get(-2)
        true_c = distribution.get(True)
        false_c = distribution.get(False)
        print(neg_1)
        if neg_1== None:
            neg_1=0
        if neg_2== None:
            neg_2=0
        if true_c== None:
            true_c=0
        if false_c==None:
            false_c=0
        number_of_rel_items[col] = len(eval_df)-neg_1
        number_of_rel_items_retreived[col] = true_c
        number_of_retrieved_items[col] = false_c + true_c
        
    return len(filtered_df), number_of_rel_items_retreived, len(simple_schema_matched)
    
def get_precision_recall(number_of_rel_items: int, number_of_rel_items_retreived: dict, number_of_retrieved_items: int):
    precision = {}
    recall = {}
    other_properties_extracted = ['PROPERTY: grain size ($\mu$m)_extracted',
                          'PROPERTY: Exp. Density (g/cm$^3$)_extracted',
                          'PROPERTY: HV_extracted',
                          'PROPERTY: Test temperature ($^\circ$C)_extracted',
                          'PROPERTY: UTS (MPa)_extracted',
                          'PROPERTY: Elongation (%)_extracted',
                          'PROPERTY: Elongation plastic (%)_extracted']
    
    for prop in number_of_rel_items_retreived.keys():
        try:
            temp_precision = number_of_rel_items_retreived[prop]/number_of_retrieved_items
        except:
            temp_precision = None
        try:
            temp_recall = number_of_rel_items_retreived[prop]/number_of_rel_items
        except:
            temp_recall = None
        precision[prop] = temp_precision
        recall[prop] = temp_recall
    pprint(precision)
    pprint(recall)



#### for one shot

In [67]:
number_of_rel_items, number_of_rel_items_retreived, number_of_retrieved_items = find_evaluation_scores('matched_simple_oneshot.csv')
print(number_of_rel_items)
print(number_of_rel_items_retreived)
print(number_of_retrieved_items)
get_precision_recall(number_of_rel_items, number_of_rel_items_retreived, number_of_retrieved_items)

Index(['Unnamed: 0', 'index', 'extracted_idx', 'original_idx',
       'match_strength', 'match_group', 'IDENTIFIER: Reference ID_original',
       'FORMULA_original', 'PROPERTY: Microstructure_original',
       'PROPERTY: Processing method_original',
       'PROPERTY: BCC/FCC/other_original',
       'PROPERTY: grain size ($\mu$m)_original',
       'PROPERTY: Exp. Density (g/cm$^3$)_original',
       'PROPERTY: Computed Density (g/cm$^3$)_original',
       'PROPERTY: HV_original', 'PROPERTY: Type of test_original',
       'PROPERTY: Test temperature ($^\circ$C)_original',
       'PROPERTY: YS (MPa)_original', 'PROPERTY: UTS (MPa)_original',
       'PROPERTY: Elongation (%)_original',
       'PROPERTY: Elongation plastic (%)_original',
       'PROPERTY: Exp. Young modulus (GPa)_original',
       'PROPERTY: Computed Young modulus (GPa)_original',
       'PROPERTY: O content (wppm)_original',
       'PROPERTY: N content (wppm)_original',
       'PROPERTY: C content (wppm)_original', 'REFER

#### for zero shot

In [68]:
number_of_rel_items, number_of_rel_items_retreived, number_of_retrieved_items = find_evaluation_scores('matched_simple_zeroshot.csv')
# print(number_of_rel_items)
# print(number_of_rel_items_retreived)
# print(number_of_retrieved_items)
get_precision_recall(number_of_rel_items, number_of_rel_items_retreived, number_of_retrieved_items)

Index(['Unnamed: 0', 'index', 'extracted_idx', 'original_idx',
       'match_strength', 'match_group', 'IDENTIFIER: Reference ID_original',
       'FORMULA_original', 'PROPERTY: Microstructure_original',
       'PROPERTY: Processing method_original',
       'PROPERTY: BCC/FCC/other_original',
       'PROPERTY: grain size ($\mu$m)_original',
       'PROPERTY: Exp. Density (g/cm$^3$)_original',
       'PROPERTY: Computed Density (g/cm$^3$)_original',
       'PROPERTY: HV_original', 'PROPERTY: Type of test_original',
       'PROPERTY: Test temperature ($^\circ$C)_original',
       'PROPERTY: YS (MPa)_original', 'PROPERTY: UTS (MPa)_original',
       'PROPERTY: Elongation (%)_original',
       'PROPERTY: Elongation plastic (%)_original',
       'PROPERTY: Exp. Young modulus (GPa)_original',
       'PROPERTY: Computed Young modulus (GPa)_original',
       'PROPERTY: O content (wppm)_original',
       'PROPERTY: N content (wppm)_original',
       'PROPERTY: C content (wppm)_original', 'REFER

#### for langchain

In [ ]:
number_of_rel_items, number_of_rel_items_retreived, number_of_retrieved_items = find_evaluation_scores('matched_langchain_simple.csv')
# print("Number of relevant items")
# print(number_of_rel_items)
# print("Number of relevant items retreived")
# print(number_of_rel_items_retreived)
# print("Number of retrieved items")
# print(number_of_retrieved_items)
get_precision_recall(number_of_rel_items, number_of_rel_items_retreived, number_of_retrieved_items)

# Complex Schema

In [ ]:
complex_schema_matched = pd.read_csv(path+'matched_complex_schema_hea.csv')

In [ ]:
complex_schema_matched.columns

Index(['Unnamed: 0', 'index', 'extracted_idx', 'original_idx',
       'match_strength', 'match_group', 'IDENTIFIER: Reference ID_original',
       'FORMULA_original', 'PROPERTY: Microstructure_original',
       'PROPERTY: Processing method_original',
       'PROPERTY: BCC/FCC/other_original',
       'PROPERTY: grain size ($\mu$m)_original',
       'PROPERTY: Exp. Density (g/cm$^3$)_original',
       'PROPERTY: Computed Density (g/cm$^3$)_original',
       'PROPERTY: HV_original', 'PROPERTY: Type of test_original',
       'PROPERTY: Test temperature ($^\circ$C)_original',
       'PROPERTY: YS (MPa)_original', 'PROPERTY: UTS (MPa)_original',
       'PROPERTY: Elongation (%)_original',
       'PROPERTY: Elongation plastic (%)_original',
       'PROPERTY: Exp. Young modulus (GPa)_original',
       'PROPERTY: Computed Young modulus (GPa)_original',
       'PROPERTY: O content (wppm)_original',
       'PROPERTY: N content (wppm)_original',
       'PROPERTY: C content (wppm)_original', 'REFER

In [ ]:
other_properties_extracted = ['PROPERTY: Microstructure_extracted',
                            'PROPERTY: Processing method_extracted',
                            'PROPERTY: BCC/FCC/other_extracted',
                            'PROPERTY: grain size ($\mu$m)_extracted',
                            'PROPERTY: Exp. Density (g/cm$^3$)_extracted', 'PROPERTY: HV_extracted',
                            'PROPERTY: Type of test_extracted',
                            'PROPERTY: Test temperature ($^\circ$C)_extracted',
                            'PROPERTY: YS (MPa)_extracted', 'PROPERTY: UTS (MPa)_extracted',
                            'PROPERTY: Elongation (%)_extracted',
                            'PROPERTY: Elongation plastic (%)_extracted',
                            'PROPERTY: Exp. Young modulus (GPa)_extracted',
                            'PROPERTY: O content (wppm)_extracted',
                            'PROPERTY: N content (wppm)_extracted',
                            'PROPERTY: C content (wppm)_extracted']
original_properties = [prop.replace('extracted', 'original') for prop in other_properties_extracted]

In [ ]:
print(original_properties)

['PROPERTY: Microstructure_original', 'PROPERTY: Processing method_original', 'PROPERTY: BCC/FCC/other_original', 'PROPERTY: grain size ($\\mu$m)_original', 'PROPERTY: Exp. Density (g/cm$^3$)_original', 'PROPERTY: HV_original', 'PROPERTY: Type of test_original', 'PROPERTY: Test temperature ($^\\circ$C)_original', 'PROPERTY: YS (MPa)_original', 'PROPERTY: UTS (MPa)_original', 'PROPERTY: Elongation (%)_original', 'PROPERTY: Elongation plastic (%)_original', 'PROPERTY: Exp. Young modulus (GPa)_original', 'PROPERTY: O content (wppm)_original', 'PROPERTY: N content (wppm)_original', 'PROPERTY: C content (wppm)_original']


In [ ]:
list_row_match = []
import numpy as np
import math

string_props = ['PROPERTY: Microstructure',
                'PROPERTY: Processing method',
                'PROPERTY: BCC/FCC/other',
                'PROPERTY: Type of test']

for i, row in complex_schema_matched.iterrows():
  row_value_extracted = {prop.replace('_extracted', ''):row[prop] for prop in other_properties_extracted}
  row_value_original = {prop.replace('_original', ''):row[prop] for prop in original_properties}
  # print(row_value_original)
  match_dict = {}
  for k_org, v_org in row_value_original.items():
    v_ext = row_value_extracted[k_org]
    if k_org in string_props:

      try:
        if type(v_org)!=str and math.isnan(v_org):
          match_dict[k_org] = -1
        elif type(v_ext)!=str and math.isnan(v_ext):
          match_dict[k_org] = -2
        if v_org==v_ext:
          match_dict[k_org] = True
        else:
          match_dict[k_org] = False
      except:
        print("Error")
        print(v_ext)
    else:
      try:
        if '>' in v_ext or '%' in v_ext:
          v_ext = v_ext.replace('>', '').replace('%','')
        v_ext = float(v_ext)
      except Exception as e:
        # print(k_org)
        pass
      if math.isnan(v_org):
        match_dict[k_org] = -1 # no valid entry
        continue
      elif type(v_ext)!=float or math.isnan(v_ext)==True:
        match_dict[k_org] = -2 # valid entry present but nothing extracted
        continue
      else:
        if v_org==v_ext:
          match_dict[k_org] = True # match
        elif v_org != v_ext:
          match_dict[k_org] = False # no match
  list_row_match.append(match_dict)

In [ ]:
print(list_row_match)

[{'PROPERTY: Microstructure': False, 'PROPERTY: Processing method': False, 'PROPERTY: BCC/FCC/other': True, 'PROPERTY: grain size ($\\mu$m)': -1, 'PROPERTY: Exp. Density (g/cm$^3$)': -1, 'PROPERTY: HV': True, 'PROPERTY: Type of test': False, 'PROPERTY: Test temperature ($^\\circ$C)': -2, 'PROPERTY: YS (MPa)': True, 'PROPERTY: UTS (MPa)': -1, 'PROPERTY: Elongation (%)': -2, 'PROPERTY: Elongation plastic (%)': -1, 'PROPERTY: Exp. Young modulus (GPa)': -1, 'PROPERTY: O content (wppm)': -1, 'PROPERTY: N content (wppm)': -1, 'PROPERTY: C content (wppm)': -1}, {'PROPERTY: Microstructure': False, 'PROPERTY: Processing method': False, 'PROPERTY: BCC/FCC/other': True, 'PROPERTY: grain size ($\\mu$m)': -1, 'PROPERTY: Exp. Density (g/cm$^3$)': -1, 'PROPERTY: HV': True, 'PROPERTY: Type of test': False, 'PROPERTY: Test temperature ($^\\circ$C)': -2, 'PROPERTY: YS (MPa)': True, 'PROPERTY: UTS (MPa)': -1, 'PROPERTY: Elongation (%)': -2, 'PROPERTY: Elongation plastic (%)': -1, 'PROPERTY: Exp. Young mo

In [ ]:
len(list_row_match)

178

In [ ]:
eval_df = pd.DataFrame(list_row_match)

In [ ]:
eval_df

,PROPERTY: Microstructure,PROPERTY: Processing method,PROPERTY: BCC/FCC/other,PROPERTY: grain size ($\mu$m),PROPERTY: Exp. Density (g/cm$^3$),PROPERTY: HV,PROPERTY: Type of test,PROPERTY: Test temperature ($^\circ$C),PROPERTY: YS (MPa),PROPERTY: UTS (MPa),PROPERTY: Elongation (%),PROPERTY: Elongation plastic (%),PROPERTY: Exp. Young modulus (GPa),PROPERTY: O content (wppm),PROPERTY: N content (wppm),PROPERTY: C content (wppm)
0,False,False,True,-1,-1,True,False,-2,True,-1,-2,-1,-1,-1,-1,-1
1,False,False,True,-1,-1,True,False,-2,True,-1,-2,-1,-1,-1,-1,-1
2,False,False,True,-1,-1,True,False,-2,True,-1,-2,-1,-1,-1,-1,-1
3,False,False,True,-1,-1,True,False,-2,True,-1,-2,-1,-1,-1,-1,-1
4,False,False,True,-1,-1,-1,False,False,True,-1,-2,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,False,False,True,-1,-1,-1,False,False,True,-2,-2,-1,-1,-1,-1,-1
174,False,False,True,-1,-1,-1,False,False,True,-2,-2,-1,-1,-1,-1,-1
175,False,False,True,-1,-1,-1,False,-2,True,-2,-1,-1,-1,-1,-1,-1
176,False,False,True,-1,-1,-1,False,-2,True,-2,-1,-1,-1,-1,-1,-1


In [ ]:
number_of_rel_items = {}
number_of_rel_items_retreived = {}
number_of_retrieved_items = {}

for col in eval_df.columns:
  distribution = eval_df[col].value_counts().to_dict()
  neg_1 = distribution.get(-1)
  neg_2 = distribution.get(-2)
  true_c = distribution.get(True)
  false_c = distribution.get(False)

  if neg_1== None:
    neg_1=0
  if neg_2== None:
    neg_2=0
  if true_c== None:
    true_c=0
  if false_c==None:
    false_c=0
  number_of_rel_items[col] = len(eval_df)-neg_1
  number_of_rel_items_retreived[col] = true_c
  number_of_retrieved_items[col] = false_c+true_c

In [ ]:
number_of_rel_items

{'PROPERTY: Microstructure': 178,
 'PROPERTY: Processing method': 178,
 'PROPERTY: BCC/FCC/other': 178,
 'PROPERTY: grain size ($\\mu$m)': 35,
 'PROPERTY: Exp. Density (g/cm$^3$)': 19,
 'PROPERTY: HV': 65,
 'PROPERTY: Type of test': 178,
 'PROPERTY: Test temperature ($^\\circ$C)': 178,
 'PROPERTY: YS (MPa)': 178,
 'PROPERTY: UTS (MPa)': 77,
 'PROPERTY: Elongation (%)': 121,
 'PROPERTY: Elongation plastic (%)': 8,
 'PROPERTY: Exp. Young modulus (GPa)': 13,
 'PROPERTY: O content (wppm)': 7,
 'PROPERTY: N content (wppm)': 5,
 'PROPERTY: C content (wppm)': 5}

In [ ]:
number_of_rel_items_retreived

{'PROPERTY: Microstructure': 68,
 'PROPERTY: Processing method': 4,
 'PROPERTY: BCC/FCC/other': 126,
 'PROPERTY: grain size ($\\mu$m)': 2,
 'PROPERTY: Exp. Density (g/cm$^3$)': 9,
 'PROPERTY: HV': 9,
 'PROPERTY: Type of test': 0,
 'PROPERTY: Test temperature ($^\\circ$C)': 62,
 'PROPERTY: YS (MPa)': 178,
 'PROPERTY: UTS (MPa)': 2,
 'PROPERTY: Elongation (%)': 7,
 'PROPERTY: Elongation plastic (%)': 6,
 'PROPERTY: Exp. Young modulus (GPa)': 5,
 'PROPERTY: O content (wppm)': 2,
 'PROPERTY: N content (wppm)': 0,
 'PROPERTY: C content (wppm)': 0}

In [ ]:
number_of_retrieved_items

{'PROPERTY: Microstructure': 178,
 'PROPERTY: Processing method': 178,
 'PROPERTY: BCC/FCC/other': 178,
 'PROPERTY: grain size ($\\mu$m)': 4,
 'PROPERTY: Exp. Density (g/cm$^3$)': 9,
 'PROPERTY: HV': 21,
 'PROPERTY: Type of test': 178,
 'PROPERTY: Test temperature ($^\\circ$C)': 105,
 'PROPERTY: YS (MPa)': 178,
 'PROPERTY: UTS (MPa)': 2,
 'PROPERTY: Elongation (%)': 8,
 'PROPERTY: Elongation plastic (%)': 6,
 'PROPERTY: Exp. Young modulus (GPa)': 5,
 'PROPERTY: O content (wppm)': 2,
 'PROPERTY: N content (wppm)': 0,
 'PROPERTY: C content (wppm)': 0}

In [ ]:
get_precision_recall(number_of_rel_items, number_of_rel_items_retreived, number_of_retrieved_items)

{'PROPERTY: BCC/FCC/other': 0.7078651685393258,
 'PROPERTY: C content (wppm)': None,
 'PROPERTY: Elongation (%)': 0.875,
 'PROPERTY: Elongation plastic (%)': 1.0,
 'PROPERTY: Exp. Density (g/cm$^3$)': 1.0,
 'PROPERTY: Exp. Young modulus (GPa)': 1.0,
 'PROPERTY: HV': 0.42857142857142855,
 'PROPERTY: Microstructure': 0.38202247191011235,
 'PROPERTY: N content (wppm)': None,
 'PROPERTY: O content (wppm)': 1.0,
 'PROPERTY: Processing method': 0.02247191011235955,
 'PROPERTY: Test temperature ($^\\circ$C)': 0.5904761904761905,
 'PROPERTY: Type of test': 0.0,
 'PROPERTY: UTS (MPa)': 1.0,
 'PROPERTY: YS (MPa)': 1.0,
 'PROPERTY: grain size ($\\mu$m)': 0.5}
{'PROPERTY: BCC/FCC/other': 0.7078651685393258,
 'PROPERTY: C content (wppm)': 0.0,
 'PROPERTY: Elongation (%)': 0.05785123966942149,
 'PROPERTY: Elongation plastic (%)': 0.75,
 'PROPERTY: Exp. Density (g/cm$^3$)': 0.47368421052631576,
 'PROPERTY: Exp. Young modulus (GPa)': 0.38461538461538464,
 'PROPERTY: HV': 0.13846153846153847,
 'PROPERT